In [4]:
# Import libraries
import torch
import numpy as np

# Functions to load and pre-process the images:
from skimage.io import imread
from skimage import img_as_ubyte
from sigver.preprocessing.normalize import preprocess_signature

# Functions to load the CNN model
from sigver.featurelearning.models import SigNet

# Functions for plotting:
import matplotlib.pyplot as plt

In [5]:
# Function to load signature
def load_signature(path):
    return img_as_ubyte(imread(path, as_gray=True))

In [6]:
# Create dictionary of signatures with key as user number
user_sigs = {}
for i in range(1, 35):
    user_sigs[i]= [load_signature('data/IISER_Genuine/Genuine/{}-G-{}.jpg'.format(i, j)) for j in  [1,2,3]]

In [7]:
# Create dictionary of preprocessed signatures with key as user number
processed_user_sigs = {}
canvas_size_iiser = (5000, 5000)
for i in range(1, 35):
    processed_user_sigs[i] = torch.tensor([preprocess_signature(sig, canvas_size_iiser) for sig in user_sigs[i]])

/tmp/ipykernel_39443/2267855104.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343904639/work/torch/csrc/utils/tensor_new.cpp:245.)
  processed_user_sigs[i] = torch.tensor([preprocess_signature(sig, canvas_size_iiser) for sig in user_sigs[i]])


In [8]:
# Convert images to appropriate dimensions, and divide by 255 to convert intensities to the range [0, 1]
for i in range(1, 35):
    processed_user_sigs[i] = processed_user_sigs[i].view(-1, 1, 150, 220).float().div(255)

### Using the CNN to obtain the feature representations

In [9]:
# If GPU is available, use it:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: {}'.format(device))

Using device: cpu


In [10]:
# Load the model
state_dict, _, _ = torch.load('models/signet.pth')
base_model = SigNet().to(device).eval()
base_model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
# Convert each image to its feature vector representation using the pretrained model.
user_features = {}
with torch.no_grad():
    for i in range(1, 35):
        user_features[i] = base_model(processed_user_sigs[i].to(device))

In [12]:
# Check the dimensions of the vector. It should be a 248-dimensional vecotr
user_features[1][0].shape

torch.Size([2048])

In [13]:
# Form the dataset and labels using the feature vectors. The labels are the user numbers from 1 to 34.
X = []
y = []
for i in range(1, 35):
    for j in range(3):
        X.append(np.array(user_features[i][j]))
        y.append(i)


In [15]:
# Convert the lists to numpy arrays
X = np.vstack(X)
print(X.shape)
y = np.array(y)
print(y.shape)

(102, 2048)
(102,)


In [16]:
# Use GridSearch to obtain best hyperparameters of SVM classifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the hyperparameter grid you want to search
param_grid = {
    'C': [0.15, 0.2, 0.25, 0.3],  # Different values for the regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel function
    'gamma': [0.01, 0.1, 1],  # Kernel coefficient for 'rbf' kernel
    'degree': [2, 3, 4],  # Degree of the polynomial kernel
    'coef0': [0.0, 1.0],  # Independent term in kernel function
    'shrinking': [True, False],  # Whether to use shrinking heuristic
    'class_weight': [None, 'balanced'],  # Class weights
}

# Create an SVM classifier
svc = SVC()

# Create a GridSearchCV object with cross-validation
grid_search = GridSearchCV(svc, param_grid, cv=3)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

print(best_params)

best_classifier = grid_search.best_estimator_

y_pred = best_classifier.predict(X_test)

accuracy = best_classifier.score(X_test, y_test)
print(f'Best Model Accuracy: {accuracy}')


/home/susheel/anaconda3/envs/sigver/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


{'C': 0.15, 'class_weight': None, 'coef0': 0.0, 'degree': 2, 'gamma': 0.01, 'kernel': 'linear', 'shrinking': True}
Best Model Accuracy: 0.7619047619047619


In [17]:
# Compute mean accuracy using Leave One Out cross validation

from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Create an SVM classifier 
clf = SVC(C=0.2, kernel='linear', gamma=0.2, degree=2, shrinking=True)

# Initialize LeaveOneOut
loo = LeaveOneOut()

# Initialize variables to store results
accuracies = []

# Perform LOOCV
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the mean accuracy over all iterations
mean_accuracy = sum(accuracies) / len(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")



Mean Accuracy: 0.7745098039215687
